In [15]:
import math
import warnings

import torch
import torchinfo
from pandas.errors import SettingWithCopyWarning
from torch import nn
from torch.utils.data import DataLoader
import timm
import glob
import os
import numpy as np
import pandas as pd
from glob import glob
import tqdm
from torch.utils.data import Dataset
import albumentations as A
import pydicom
from single_dataset import DATA_PATH

In [16]:
train_merged = pd.read_csv(DATA_PATH / "temp_train.csv")
train_merged

,study_id,series_id,instance_number,condition,level,x,y,plane,label
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR,Normal/Mild
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Sagittal T2/STIR,Normal/Mild
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Sagittal T2/STIR,Normal/Mild
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Sagittal T2/STIR,Normal/Mild
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Sagittal T2/STIR,Normal/Mild
...,...,...,...,...,...,...,...,...,...
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063,Sagittal T1,Normal/Mild
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084,Sagittal T1,Normal/Mild
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624,Sagittal T1,Normal/Mild
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333,Sagittal T1,Normal/Mild


In [17]:
train_merged.loc[train_merged.label == "Moderate"]

,study_id,series_id,instance_number,condition,level,x,y,plane,label
5,4003253,1054713880,4,Right Neural Foraminal Narrowing,L4/L5,187.961759,251.839388,Sagittal T1,Moderate
7,4003253,1054713880,5,Right Neural Foraminal Narrowing,L3/L4,187.227533,210.722753,Sagittal T1,Moderate
11,4003253,1054713880,11,Left Neural Foraminal Narrowing,L4/L5,186.504472,251.592129,Sagittal T1,Moderate
21,4003253,2448190387,28,Left Subarticular Stenosis,L4/L5,172.948842,156.911197,Axial T2,Moderate
28,4646740,3201256954,22,Right Subarticular Stenosis,L2/L3,200.977376,256.868778,Axial T2,Moderate
...,...,...,...,...,...,...,...,...,...
48653,4287160193,1507070277,8,Spinal Canal Stenosis,L2/L3,369.435472,321.805360,Sagittal T2/STIR,Moderate
48659,4287160193,1820446240,9,Left Subarticular Stenosis,L4/L5,143.925926,136.296296,Axial T2,Moderate
48660,4287160193,1820446240,10,Right Subarticular Stenosis,L4/L5,116.502655,137.564602,Axial T2,Moderate
48661,4287160193,1820446240,16,Left Subarticular Stenosis,L3/L4,146.592593,149.037037,Axial T2,Moderate


In [18]:
train_merged.loc[train_merged.label == "Severe"]

,study_id,series_id,instance_number,condition,level,x,y,plane,label
31,4646740,3201256954,34,Left Subarticular Stenosis,L4/L5,222.606272,264.083624,Axial T2,Severe
48,4646740,3666319702,10,Spinal Canal Stenosis,L4/L5,241.700041,324.625668,Sagittal T2/STIR,Severe
184,13317052,1539051863,12,Left Neural Foraminal Narrowing,L5/S1,210.574338,258.867617,Sagittal T1,Severe
216,22191399,3687121182,37,Right Subarticular Stenosis,L4/L5,154.307911,158.636776,Axial T2,Severe
223,22191399,3753885158,10,Spinal Canal Stenosis,L4/L5,254.918077,315.437500,Sagittal T2/STIR,Severe
...,...,...,...,...,...,...,...,...,...
48626,4284048608,1875151370,17,Left Neural Foraminal Narrowing,L5/S1,209.969231,344.861538,Sagittal T1,Severe
48630,4284048608,3767301072,12,Spinal Canal Stenosis,L4/L5,280.208006,391.083203,Sagittal T2/STIR,Severe
48638,4284048608,3891595425,30,Left Subarticular Stenosis,L4/L5,245.587838,210.162162,Axial T2,Severe
48639,4284048608,3891595425,30,Right Subarticular Stenosis,L4/L5,214.767884,210.021309,Axial T2,Severe


In [19]:
dist = train_merged[["plane", "condition", "level", "instance_number", "label"]].groupby(["plane", "condition", "level", "label"]).count().reset_index()
dist

,plane,condition,level,label,instance_number
0,Axial T2,Left Subarticular Stenosis,L1/L2,Moderate,93
1,Axial T2,Left Subarticular Stenosis,L1/L2,Normal/Mild,1688
2,Axial T2,Left Subarticular Stenosis,L1/L2,Severe,28
3,Axial T2,Left Subarticular Stenosis,L2/L3,Moderate,254
4,Axial T2,Left Subarticular Stenosis,L2/L3,Normal/Mild,1554
...,...,...,...,...,...
75,Sagittal T2/STIR,Spinal Canal Stenosis,L4/L5,Normal/Mild,1481
76,Sagittal T2/STIR,Spinal Canal Stenosis,L4/L5,Severe,253
77,Sagittal T2/STIR,Spinal Canal Stenosis,L5/S1,Moderate,51
78,Sagittal T2/STIR,Spinal Canal Stenosis,L5/S1,Normal/Mild,1899


In [20]:

g = train_merged.groupby(["plane", "condition", "level", "label"])
g = g.apply(lambda x: x.sample(250, replace=len(x) <= 250).reset_index(drop=True))
data = g.reset_index(drop=True)
data

,study_id,series_id,instance_number,condition,level,x,y,plane,label
0,3239069834,2533594621,13,Left Subarticular Stenosis,L1/L2,287.888502,244.181185,Axial T2,Moderate
1,2661178959,1663050763,2,Left Subarticular Stenosis,L1/L2,146.350442,131.511504,Axial T2,Moderate
2,1743493727,419554762,23,Left Subarticular Stenosis,L1/L2,259.992628,292.571429,Axial T2,Moderate
3,1546104377,2164250177,8,Left Subarticular Stenosis,L1/L2,134.852033,98.253659,Axial T2,Moderate
4,2351425784,4040614859,13,Left Subarticular Stenosis,L1/L2,260.797127,332.708080,Axial T2,Moderate
...,...,...,...,...,...,...,...,...,...
19995,2975955864,3339880455,7,Spinal Canal Stenosis,L5/S1,421.373237,608.932709,Sagittal T2/STIR,Severe
19996,2530679352,3989445153,9,Spinal Canal Stenosis,L5/S1,354.857143,444.285714,Sagittal T2/STIR,Severe
19997,1302048123,3767852858,10,Spinal Canal Stenosis,L5/S1,219.942029,350.811594,Sagittal T2/STIR,Severe
19998,2746735779,497960577,9,Spinal Canal Stenosis,L5/S1,327.827514,419.661275,Sagittal T2/STIR,Severe


In [21]:
data[["plane", "condition", "level", "instance_number", "label"]].groupby(["plane", "condition", "level", "label"]).count().reset_index()

,plane,condition,level,label,instance_number
0,Axial T2,Left Subarticular Stenosis,L1/L2,Moderate,250
1,Axial T2,Left Subarticular Stenosis,L1/L2,Normal/Mild,250
2,Axial T2,Left Subarticular Stenosis,L1/L2,Severe,250
3,Axial T2,Left Subarticular Stenosis,L2/L3,Moderate,250
4,Axial T2,Left Subarticular Stenosis,L2/L3,Normal/Mild,250
...,...,...,...,...,...
75,Sagittal T2/STIR,Spinal Canal Stenosis,L4/L5,Normal/Mild,250
76,Sagittal T2/STIR,Spinal Canal Stenosis,L4/L5,Severe,250
77,Sagittal T2/STIR,Spinal Canal Stenosis,L5/S1,Moderate,250
78,Sagittal T2/STIR,Spinal Canal Stenosis,L5/S1,Normal/Mild,250


In [24]:
from sklearn.model_selection import KFold

skf = KFold(n_splits=14, shuffle=True, random_state=8620)
fold_score = []
study_ids = np.array(train_merged.study_id.unique())

df_train = pd.DataFrame()
df_valid = pd.DataFrame()
for fold, (trn_idx, val_idx) in enumerate(skf.split(range(len(study_ids)))):
    print("train size", len(trn_idx), "test size", len(val_idx))
    trx_study_id = study_ids[trn_idx]
    val_study_id = study_ids[val_idx]

    df_train = train_merged.loc[train_merged.study_id.isin(trx_study_id)]
    df_valid = train_merged.loc[train_merged.study_id.isin(val_study_id)]
    break

train size 1833 test size 141


In [28]:
g = df_train.groupby(["plane", "condition", "level", "label"])
g = g.apply(lambda x: x.sample(250, replace=len(x) <= 250, random_state=8620).reset_index(drop=True))
data = g.reset_index(drop=True)
data

,study_id,series_id,instance_number,condition,level,x,y,plane,label
0,264945797,3695845416,3,Left Subarticular Stenosis,L1/L2,141.524528,122.520755,Axial T2,Moderate
1,4177419629,3147720892,9,Left Subarticular Stenosis,L1/L2,216.633962,208.301887,Axial T2,Moderate
2,953639220,3014494618,23,Left Subarticular Stenosis,L1/L2,280.014159,268.007080,Axial T2,Moderate
3,1546104377,2164250177,8,Left Subarticular Stenosis,L1/L2,134.852033,98.253659,Axial T2,Moderate
4,3192842688,1330627813,18,Left Subarticular Stenosis,L1/L2,176.344175,214.813774,Axial T2,Moderate
...,...,...,...,...,...,...,...,...,...
19995,3882759508,963820603,11,Spinal Canal Stenosis,L5/S1,246.185538,345.158059,Sagittal T2/STIR,Severe
19996,1302048123,3767852858,10,Spinal Canal Stenosis,L5/S1,219.942029,350.811594,Sagittal T2/STIR,Severe
19997,2975955864,3339880455,7,Spinal Canal Stenosis,L5/S1,421.373237,608.932709,Sagittal T2/STIR,Severe
19998,1902377008,3170338765,8,Spinal Canal Stenosis,L5/S1,494.782301,576.679646,Sagittal T2/STIR,Severe


In [29]:
data[["plane", "condition", "level", "instance_number", "label"]].groupby(["plane", "condition", "level", "label"]).count().reset_index()

,plane,condition,level,label,instance_number
0,Axial T2,Left Subarticular Stenosis,L1/L2,Moderate,250
1,Axial T2,Left Subarticular Stenosis,L1/L2,Normal/Mild,250
2,Axial T2,Left Subarticular Stenosis,L1/L2,Severe,250
3,Axial T2,Left Subarticular Stenosis,L2/L3,Moderate,250
4,Axial T2,Left Subarticular Stenosis,L2/L3,Normal/Mild,250
...,...,...,...,...,...
75,Sagittal T2/STIR,Spinal Canal Stenosis,L4/L5,Normal/Mild,250
76,Sagittal T2/STIR,Spinal Canal Stenosis,L4/L5,Severe,250
77,Sagittal T2/STIR,Spinal Canal Stenosis,L5/S1,Moderate,250
78,Sagittal T2/STIR,Spinal Canal Stenosis,L5/S1,Normal/Mild,250


In [39]:
new_data = data.apply(lambda row: df_train.loc[(df_train.study_id == row.study_id) & (df_train.series_id == row.series_id) & (df_train.instance_number == row.instance_number)], axis=1).tolist()
new_data = pd.concat(new_data, ignore_index=True)
new_data

,study_id,series_id,instance_number,condition,level,x,y,plane,label
0,264945797,3695845416,3,Left Subarticular Stenosis,L1/L2,141.524528,122.520755,Axial T2,Moderate
1,264945797,3695845416,3,Right Subarticular Stenosis,L1/L2,121.883186,122.569912,Axial T2,Moderate
2,4177419629,3147720892,9,Left Subarticular Stenosis,L1/L2,216.633962,208.301887,Axial T2,Moderate
3,4177419629,3147720892,9,Right Subarticular Stenosis,L1/L2,180.078325,215.705128,Axial T2,Normal/Mild
4,953639220,3014494618,23,Left Subarticular Stenosis,L1/L2,280.014159,268.007080,Axial T2,Moderate
...,...,...,...,...,...,...,...,...,...
55395,3867046855,35950467,8,Spinal Canal Stenosis,L1/L2,269.167975,225.054945,Sagittal T2/STIR,Normal/Mild
55396,3867046855,35950467,8,Spinal Canal Stenosis,L2/L3,251.485086,280.514914,Sagittal T2/STIR,Normal/Mild
55397,3867046855,35950467,8,Spinal Canal Stenosis,L3/L4,235.409733,329.544741,Sagittal T2/STIR,Normal/Mild
55398,3867046855,35950467,8,Spinal Canal Stenosis,L4/L5,236.953947,379.368421,Sagittal T2/STIR,Severe


In [40]:
new_data.drop_duplicates(subset=["study_id", "series_id", "instance_number", "condition", "level", "label"], keep="first", inplace=True)

In [41]:
new_data

,study_id,series_id,instance_number,condition,level,x,y,plane,label
0,264945797,3695845416,3,Left Subarticular Stenosis,L1/L2,141.524528,122.520755,Axial T2,Moderate
1,264945797,3695845416,3,Right Subarticular Stenosis,L1/L2,121.883186,122.569912,Axial T2,Moderate
2,4177419629,3147720892,9,Left Subarticular Stenosis,L1/L2,216.633962,208.301887,Axial T2,Moderate
3,4177419629,3147720892,9,Right Subarticular Stenosis,L1/L2,180.078325,215.705128,Axial T2,Normal/Mild
4,953639220,3014494618,23,Left Subarticular Stenosis,L1/L2,280.014159,268.007080,Axial T2,Moderate
...,...,...,...,...,...,...,...,...,...
54567,2621581337,1393020403,8,Spinal Canal Stenosis,L4/L5,161.969904,196.990424,Sagittal T2/STIR,Normal/Mild
54568,2621581337,1393020403,8,Spinal Canal Stenosis,L5/S1,172.038304,218.878249,Sagittal T2/STIR,Severe
54594,2746735779,497960577,9,Spinal Canal Stenosis,L4/L5,310.199971,382.803685,Sagittal T2/STIR,Severe
54595,2746735779,497960577,9,Spinal Canal Stenosis,L5/S1,327.827514,419.661275,Sagittal T2/STIR,Severe
